<a href="https://colab.research.google.com/github/vlimax/API_Rest_do_zero_com_colab/blob/main/API_Restfull_Gemini_Collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.0 Considerações iniciais
Esse guia foi feito com base no arquivo original do google sobre o assunto e estudos sobre o tema. O arquivo original do google você pode conferir fazendo esses passos:
1. Abrir notebook
2. Em github pesquise: google/generative-ai-docs/site
3. Clica nesse endereço: en/gemini-api/docs/get-started/python.ipynb

No geral, esse notebook contém o básico em português-br com minha didática🤗, feito para ajudar colegas que estão aprendendo o mesmo assunto.

## 2.0 Breve Indrodução
1. O Google fornece SDKs (Software Development Kits), que são 'kits de ferramentas 🛠️' oficiais usadas no desenvolvimento de aplicações e conectar softwares às suas plataformas. Para IA generativa com Python🐍, usamos a biblioteca google-generativeai. A principal vantagem de SDKs é a abstração🧠: não precisamos construir requisições web complexas para acessar API (API REST) do zero. O SDK já traz funções prontas (methods) para isso, tornando o código modular e simples. Na prática, usa-se como qualquer biblioteca (import ...), mas com o propósito específico de usar os serviços do Google.
2. Para praticar com APIs no colab 💪, vamos precisar apenas do SDK google-generativeai, da biblioteca do colab que já vem instalada no colab e de outras bibliotecas auxiliares que não serão o foco aqui. Também vou considerar que você entende o que é preciso para acessar uma API (a chave) e práticas de segurança, quais são os parâmetros que você pode configurar numa LLM (max_tokens, temperature, top-k, etc) e que possui noções de programação (e mesmo se vc não tiver, só executar o código seguindo as intruções de uso😊).

### 3.0 Instala o Pacote SDK de IA generativa do Google.

In [4]:
!pip install -q -U google-generativeai

### 4.0 Importa Bibliotecas

In [5]:
import google.generativeai as genai #IA Generativa
from google.colab import userdata #Acesso ao colab.

#Pacotes de Formatação (não aprofundarei neles)
import textwrap
from IPython.display import Markdown

### 5.0 Configuração de Chave
1. Vá em https://aistudio.google.com/api-keys, crie a chave e copie.
2. No colab, crie uma variável de ambiente clicando no símbolo de "🔑" na esquerda do seu monitor. Cole a chave em 'value' e o nome deve ser igual ao que vc colocar no lugar de GEMINI_API_KEY entre aspas no código abaixo. Não esquece de permitir acesso do notebook à variável de ambiente que você criou.

In [6]:
GOOGLE_API_KEY = userdata.get("GEMINI_API_KEY")

genai.configure(api_key=GOOGLE_API_KEY)

### 6.0 Configurações para Formatação
A função abaixo vai formatar melhor a respota, para deixar mais "limpa" visualmente.

In [10]:
def to_markdown(text):
    text = text.replace("•", "  *")
    return Markdown(textwrap.indent(text, "> ", predicate=lambda _: True))

### 7.0 Escolher Modelo
Aqui vai um resumo sobre cada modelo disponível no momento que escrevi esse notebook, pode ajudar na sua escolha.

1. Tarefas Complexas (Raciocínio Matemático, Lógica Pesada, Programação Avançada)

Use os modelos Pro. Eles têm mais parâmetros e "pensam" melhor antes de responder.

    Modelos: gemini-3-pro-preview, gemini-2.5-pro, deep-research-pro-preview (para pesquisas extensas).

    Uso: Provar teoremas, gerar códigos complexos em Python, analisar grandes volumes de dados não estruturados.

2. Tarefas Intermediárias (O "Padrão" / Custo-Benefício)

Use os modelos Flash. São rápidos, baratos e inteligentes o suficiente para 90% das tarefas.

    Modelos: gemini-3-flash-preview, gemini-2.5-flash, gemini-2.0-flash.

    Uso: RAG (seu curso atual), chatbots, resumos de textos, extração de dados de imagens (OCR).

3. Tarefas Simples ou Alta Velocidade (Classificação, Respostas Curtas)

Use os modelos Lite, Nano ou Gemma (versões pequenas).

    Modelos: gemini-2.5-flash-lite, gemini-2.0-flash-lite, nano-banana-pro.

    Gemma (Open Models): gemma-3-1b a gemma-3-4b são para tarefas triviais e rápidas. O gemma-3-27b compete com o Flash.

    Uso: Classificar sentimentos (positivo/negativo), formatar JSON, converter unidades.

4. Modelos Especialistas (Sufixos indicam a função)

Não use para texto geral, use apenas para a função específica:

    -image / nano-banana: Apenas para gerar ou editar imagens.

    -tts: Text-to-Speech (converter texto em áudio).

    -computer-use / -robotics: Para agentes autônomos que controlam mouse/teclado ou robôs.

In [7]:
#Execute esse bloco APENAS se quiser ver a lista completa dos modelos.
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-flash-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-use-preview-10-2025
models/deep-research-pro-preview-12-2025


Digite o nome do modelo entre as aspas.

In [8]:
model = genai.GenerativeModel("gemini-2.5-flash")

### 8.0 Enviar Pergunta
Coloque a pergunta que você quer fazer entre as aspas.

In [9]:
%%time
response = model.generate_content(
    "Explique passo a passo, com 3 exemplos, a diferença entre classe e objeto em pyhton, com a síntaxe de cada um e de maniera didática."
    )

CPU times: user 200 ms, sys: 24.3 ms, total: 224 ms
Wall time: 24.7 s


### 9.0 Exibir Resposta

In [11]:
to_markdown(response.text)
#Teste print(response.txt) para ver a diferença sem a formatação.

> Com certeza! Vamos desmistificar a diferença entre Classe e Objeto em Python de uma maneira bem didática, usando analogias e exemplos práticos.
> 
> ---
> 
> ## Classe vs. Objeto em Python: A Fábrica e os Produtos
> 
> Imagine que você quer construir vários carros. Você não constrói cada carro do zero toda vez. Em vez disso, você usa um **projeto** (uma planta, um molde) que especifica como um carro deve ser feito: quais peças ele tem, como ele funciona, etc.
> 
> *   Essa **planta, projeto ou molde** é o que chamamos de **CLASSE** em Python.
> *   Cada **carro real e físico** que você constrói a partir desse projeto é um **OBJETO** (ou instância) em Python.
> 
> ### A Analogia do Cortador de Biscoitos
> 
> A melhor analogia é a do **cortador de biscoitos**:
> 
> *   O **cortador de biscoitos** (o molde de metal ou plástico) é a **CLASSE**. Ele define a forma, mas não é um biscoito comestível por si só.
> *   Cada **biscoito individual** que você faz usando esse cortador é um **OBJETO**. Cada biscoito é uma entidade real, tem sua própria consistência, sabor, etc. Você pode ter vários biscoitos do mesmo formato, mas são todos objetos separados.
> 
> ---
> 
> ### O Que é uma **CLASSE**?
> 
> Em Python, uma **classe** é um **modelo**, um **projeto** ou uma **planta** para criar objetos. Ela define as características (chamadas **atributos**) e os comportamentos (chamados **métodos**) que os objetos criados a partir dela terão.
> 
> *   **Atributos:** São como as variáveis que pertencem a um objeto. Descrevem o "estado" do objeto (ex: um carro tem cor, marca, modelo).
> *   **Métodos:** São como funções que pertencem a um objeto. Descrevem o "comportamento" ou as "ações" que o objeto pode realizar (ex: um carro pode acelerar, frear).
> 
> **Sintaxe de uma Classe:**
> 
> ```python
> class NomeDaClasse:
>     # Atributos (características) e Métodos (comportamentos) são definidos aqui
>     pass # 'pass' é usado quando a classe está vazia, apenas como placeholder
> ```
> 
> ---
> 
> ### O Que é um **OBJETO**?
> 
> Um **objeto** é uma **instância concreta** de uma classe. É uma "coisa real" que foi criada a partir do modelo definido pela classe. Quando você cria um objeto, ele ocupa um espaço na memória e tem seus próprios valores para os atributos e pode executar os métodos definidos na classe.
> 
> *   Você pode criar **vários objetos** a partir da mesma classe, e cada um terá seu próprio conjunto de dados (seus próprios valores para os atributos), mas todos seguirão a mesma estrutura e terão os mesmos comportamentos definidos na classe.
> 
> **Sintaxe para Criar um Objeto (Instanciar uma Classe):**
> 
> ```python
> nome_do_objeto = NomeDaClasse() # Chamamos a classe como se fosse uma função
> ```
> 
> ---
> 
> ### Resumo das Diferenças:
> 
> | Característica | CLASSE                                      | OBJETO (Instância)                               |
> | :------------- | :------------------------------------------ | :----------------------------------------------- |
> | **Definição**  | Modelo, projeto, planta.                    | Entidade real, item concreto.                    |
> | **Criação**    | Definida uma vez.                           | Pode-se criar muitos a partir de uma classe.     |
> | **Ocupa memória?** | Não, apenas define a estrutura.             | Sim, ocupa memória com seus próprios dados.      |
> | **Exemplo**    | O projeto de um "Carro".                    | O meu carro vermelho, o carro azul do vizinho.   |
> | **Sintaxe**    | `class Carro:`                              | `meu_carro = Carro()`                            |
> 
> ---
> 
> ## Passo a Passo com 3 Exemplos Práticos:
> 
> Vamos construir exemplos com níveis crescentes de complexidade.
> 
> ### Exemplo 1: Pessoa (Conceito Básico)
> 
> Neste exemplo, teremos uma classe `Pessoa` com atributos básicos.
> 
> **Passo 1: Definir a Classe `Pessoa`**
> 
> Vamos criar um molde para pessoas. Por enquanto, elas terão apenas um nome e uma idade.
> A convenção em Python é que o nome da classe começa com letra maiúscula.
> 
> ```python
> # 1. Definição da CLASSE 'Pessoa'
> class Pessoa:
>     # O método especial __init__ (construtor) é executado
>     # sempre que um novo objeto 'Pessoa' é criado.
>     # 'self' se refere ao próprio objeto que está sendo criado.
>     def __init__(self, nome, idade):
>         self.nome = nome  # Atributo 'nome' do objeto
>         self.idade = idade # Atributo 'idade' do objeto
> 
>     # Um método (comportamento) que a Pessoa pode fazer
>     def apresentar(self):
>         print(f"Olá, meu nome é {self.nome} e tenho {self.idade} anos.")
> 
> # Fim da definição da CLASSE Pessoa
> ```
> 
> **Passo 2: Criar Objetos (Instâncias) da Classe `Pessoa`**
> 
> Agora que temos o molde, vamos criar pessoas reais a partir dele.
> 
> ```python
> # 2. Criação de OBJETOS da classe 'Pessoa'
> 
> # Criando o primeiro objeto (instância) 'Pessoa'
> # 'alice' é um OBJETO.
> # Passamos os valores "Alice" e 30 para o método __init__.
> alice = Pessoa("Alice", 30)
> 
> # Criando o segundo objeto (instância) 'Pessoa'
> # 'bob' é um OBJETO.
> bob = Pessoa("Bob", 25)
> 
> # Fim da criação dos OBJETOS
> ```
> 
> **Passo 3: Acessar Atributos e Chamar Métodos dos Objetos**
> 
> Cada objeto tem seus próprios atributos e pode executar seus próprios métodos.
> 
> ```python
> # 3. Acessando atributos e chamando métodos dos OBJETOS
> 
> print(f"Nome da primeira pessoa: {alice.nome}") # Acessando o atributo 'nome' de 'alice'
> print(f"Idade da segunda pessoa: {bob.idade}")  # Acessando o atributo 'idade' de 'bob'
> 
> alice.apresentar() # Chamando o método 'apresentar' do objeto 'alice'
> bob.apresentar()   # Chamando o método 'apresentar' do objeto 'bob'
> 
> # Podemos ver que são objetos distintos na memória
> print(f"Objeto 'alice': {alice}")
> print(f"Objeto 'bob': {bob}")
> ```
> 
> **Saída esperada:**
> 
> ```
> Nome da primeira pessoa: Alice
> Idade da segunda pessoa: 25
> Olá, meu nome é Alice e tenho 30 anos.
> Olá, meu nome é Bob e tenho 25 anos.
> Objeto 'alice': <__main__.Pessoa object at 0x...>
> Objeto 'bob': <__main__.Pessoa object at 0x...>
> ```
> *Note os endereços de memória diferentes para `alice` e `bob`, confirmando que são objetos distintos.*
> 
> ---
> 
> ### Exemplo 2: Carro (Com Métodos Mais Elaborados)
> 
> Vamos criar uma classe `Carro` com atributos para marca, modelo e velocidade atual, e métodos para acelerar e frear.
> 
> **Passo 1: Definir a Classe `Carro`**
> 
> ```python
> # 1. Definição da CLASSE 'Carro'
> class Carro:
>     def __init__(self, marca, modelo, cor):
>         self.marca = marca
>         self.modelo = modelo
>         self.cor = cor
>         self.velocidade_atual = 0 # Todo carro começa parado
> 
>     # Método para acelerar o carro
>     def acelerar(self, incremento):
>         self.velocidade_atual += incremento
>         print(f"O {self.modelo} {self.cor} está acelerando. Velocidade atual: {self.velocidade_atual} km/h.")
> 
>     # Método para frear o carro
>     def frear(self, decremento):
>         self.velocidade_atual -= decremento
>         if self.velocidade_atual < 0:
>             self.velocidade_atual = 0
>         print(f"O {self.modelo} {self.cor} está freando. Velocidade atual: {self.velocidade_atual} km/h.")
> 
>     # Método para exibir informações do carro
>     def mostrar_info(self):
>         print(f"--- Informações do Carro ---")
>         print(f"Marca: {self.marca}")
>         print(f"Modelo: {self.modelo}")
>         print(f"Cor: {self.cor}")
>         print(f"Velocidade Atual: {self.velocidade_atual} km/h")
>         print(f"--------------------------")
> 
> # Fim da definição da CLASSE Carro
> ```
> 
> **Passo 2: Criar Objetos da Classe `Carro`**
> 
> ```python
> # 2. Criação de OBJETOS da classe 'Carro'
> 
> # Criando o primeiro objeto 'Carro'
> # 'meu_carro' é um OBJETO.
> meu_carro = Carro("Toyota", "Corolla", "Prata")
> 
> # Criando o segundo objeto 'Carro'
> # 'carro_do_vizinho' é um OBJETO.
> carro_do_vizinho = Carro("Ford", "Fiesta", "Vermelho")
> 
> # Fim da criação dos OBJETOS
> ```
> 
> **Passo 3: Interagir com os Objetos `Carro`**
> 
> ```python
> # 3. Interagindo com os OBJETOS 'Carro'
> 
> meu_carro.mostrar_info()
> meu_carro.acelerar(50)
> meu_carro.acelerar(30)
> meu_carro.frear(20)
> meu_carro.mostrar_info()
> 
> print("\n--- Carro do Vizinho ---")
> carro_do_vizinho.mostrar_info()
> carro_do_vizinho.acelerar(40)
> carro_do_vizinho.frear(10)
> carro_do_vizinho.mostrar_info()
> ```
> 
> **Saída esperada:**
> 
> ```
> --- Informações do Carro ---
> Marca: Toyota
> Modelo: Corolla
> Cor: Prata
> Velocidade Atual: 0 km/h
> --------------------------
> O Corolla Prata está acelerando. Velocidade atual: 50 km/h.
> O Corolla Prata está acelerando. Velocidade atual: 80 km/h.
> O Corolla Prata está freando. Velocidade atual: 60 km/h.
> --- Informações do Carro ---
> Marca: Toyota
> Modelo: Corolla
> Cor: Prata
> Velocidade Atual: 60 km/h
> --------------------------
> 
> --- Carro do Vizinho ---
> --- Informações do Carro ---
> Marca: Ford
> Modelo: Fiesta
> Cor: Vermelho
> Velocidade Atual: 0 km/h
> --------------------------
> O Fiesta Vermelho está acelerando. Velocidade atual: 40 km/h.
> O Fiesta Vermelho está freando. Velocidade atual: 30 km/h.
> --- Informações do Carro ---
> Marca: Ford
> Modelo: Fiesta
> Cor: Vermelho
> Velocidade Atual: 30 km/h
> --------------------------
> ```
> *Observe como cada carro mantém sua própria `velocidade_atual` e suas próprias características (cor, marca, modelo), mesmo usando os mesmos métodos da classe `Carro`.*
> 
> ---
> 
> ### Exemplo 3: Produto de Estoque (Mais Funções e Controle)
> 
> Para um sistema de estoque, vamos criar uma classe `Produto` com atributos como nome, preço, quantidade em estoque, e métodos para adicionar/remover do estoque e calcular o valor total.
> 
> **Passo 1: Definir a Classe `Produto`**
> 
> ```python
> # 1. Definição da CLASSE 'Produto'
> class Produto:
>     def __init__(self, nome, preco, quantidade_estoque):
>         self.nome = nome
>         self.preco = preco
>         # Validação simples para garantir que a quantidade não seja negativa
>         if quantidade_estoque >= 0:
>             self.quantidade_estoque = quantidade_estoque
>         else:
>             self.quantidade_estoque = 0
>             print(f"Atenção: Quantidade inicial de {nome} não pode ser negativa. Definido como 0.")
> 
>     # Método para adicionar produtos ao estoque
>     def adicionar_estoque(self, quantidade):
>         if quantidade > 0:
>             self.quantidade_estoque += quantidade
>             print(f"Adicionados {quantidade} unidades de {self.nome}. Novo estoque: {self.quantidade_estoque}.")
>         else:
>             print("Quantidade a adicionar deve ser positiva.")
> 
>     # Método para remover produtos do estoque (vender)
>     def remover_estoque(self, quantidade):
>         if quantidade > 0:
>             if self.quantidade_estoque >= quantidade:
>                 self.quantidade_estoque -= quantidade
>                 print(f"Removidas {quantidade} unidades de {self.nome}. Novo estoque: {self.quantidade_estoque}.")
>             else:
>                 print(f"Estoque insuficiente para {self.nome}. Disponível: {self.quantidade_estoque}.")
>         else:
>             print("Quantidade a remover deve ser positiva.")
> 
>     # Método para calcular o valor total do estoque deste produto
>     def calcular_valor_total(self):
>         return self.preco * self.quantidade_estoque
> 
>     # Método para exibir os detalhes do produto
>     def mostrar_detalhes(self):
>         print(f"\n--- Detalhes do Produto ---")
>         print(f"Nome: {self.nome}")
>         print(f"Preço Unitário: R${self.preco:.2f}")
>         print(f"Quantidade em Estoque: {self.quantidade_estoque}")
>         print(f"Valor Total em Estoque: R${self.calcular_valor_total():.2f}")
>         print(f"---------------------------")
> 
> # Fim da definição da CLASSE Produto
> ```
> 
> **Passo 2: Criar Objetos da Classe `Produto`**
> 
> ```python
> # 2. Criação de OBJETOS da classe 'Produto'
> 
> # 'arroz' é um OBJETO.
> arroz = Produto("Arroz Pacote 1kg", 7.99, 100)
> 
> # 'feijao' é um OBJETO.
> feijao = Produto("Feijão Carioca 1kg", 9.50, 50)
> 
> # 'refrigerante' é um OBJETO.
> refrigerante = Produto("Refrigerante 2L", 6.00, 75)
> 
> # Fim da criação dos OBJETOS
> ```
> 
> **Passo 3: Gerenciar o Estoque com os Objetos `Produto`**
> 
> ```python
> # 3. Gerenciando o estoque dos OBJETOS 'Produto'
> 
> arroz.mostrar_detalhes()
> arroz.adicionar_estoque(20)
> arroz.remover_estoque(15)
> arroz.mostrar_detalhes()
> 
> feijao.mostrar_detalhes()
> feijao.remover_estoque(60) # Tentar remover mais do que tem
> feijao.remover_estoque(30)
> feijao.adicionar_estoque(10)
> feijao.mostrar_detalhes()
> 
> refrigerante.mostrar_detalhes()
> print(f"O valor total do estoque de {refrigerante.nome} é R${refrigerante.calcular_valor_total():.2f}")
> ```
> 
> **Saída esperada:**
> 
> ```
> --- Detalhes do Produto ---
> Nome: Arroz Pacote 1kg
> Preço Unitário: R$7.99
> Quantidade em Estoque: 100
> Valor Total em Estoque: R$799.00
> ---------------------------
> Adicionados 20 unidades de Arroz Pacote 1kg. Novo estoque: 120.
> Removidas 15 unidades de Arroz Pacote 1kg. Novo estoque: 105.
> 
> --- Detalhes do Produto ---
> Nome: Arroz Pacote 1kg
> Preço Unitário: R$7.99
> Quantidade em Estoque: 105
> Valor Total em Estoque: R$838.95
> ---------------------------
> 
> --- Detalhes do Produto ---
> Nome: Feijão Carioca 1kg
> Preço Unitário: R$9.50
> Quantidade em Estoque: 50
> Valor Total em Estoque: R$475.00
> ---------------------------
> Estoque insuficiente para Feijão Carioca 1kg. Disponível: 50.
> Removidas 30 unidades de Feijão Carioca 1kg. Novo estoque: 20.
> Adicionados 10 unidades de Feijão Carioca 1kg. Novo estoque: 30.
> 
> --- Detalhes do Produto ---
> Nome: Feijão Carioca 1kg
> Preço Unitário: R$9.50
> Quantidade em Estoque: 30
> Valor Total em Estoque: R$285.00
> ---------------------------
> 
> --- Detalhes do Produto ---
> Nome: Refrigerante 2L
> Preço Unitário: R$6.00
> Quantidade em Estoque: 75
> Valor Total em Estoque: R$450.00
> ---------------------------
> O valor total do estoque de Refrigerante 2L é R$450.00
> ```
> *Cada produto (`arroz`, `feijao`, `refrigerante`) é um objeto independente, gerenciando seu próprio estoque e informações. Eles compartilham os mesmos métodos (adicionar, remover, calcular), mas atuam sobre seus próprios dados.*
> 
> ---
> 
> ### Conclusão
> 
> Espero que esta explicação detalhada, com as analogias e os exemplos passo a passo, tenha tornado a diferença entre classe e objeto em Python muito mais clara! Em resumo:
> 
> *   **Classe:** O projeto, o molde, a definição de como algo deve ser.
> *   **Objeto:** A coisa real e concreta, criada a partir do projeto da classe, com suas próprias características e capacidade de realizar ações.
> 
> Dominar esses conceitos é fundamental para programar em Python e entender o paradigma de Programação Orientada a Objetos (POO).